In [1]:
import requests
from requests.adapters import HTTPAdapter, Retry

from tqdm import tqdm
import math
import itertools
import json
import time
import os


def get_events(league = 'NBA', start = "2022-09-09", end = "9999-12-31"):
    '''
    Get all league events on stubhub.
    '''

    event_category = {'NBA':{'path':'nba-tickets','categoryid': "6453"},
                      'NFL':{'path':'nfl-tickets','categoryid': "5084"}}


    url = f"https://www.stubhub.com/{event_category[league]['path']}"

    querystring = {"pageIndex":"0",
                "gridFilterType":"0",
                "sortBy":"0",
                "method":"GetFilteredEvents",
                "categoryId": event_category[league]['categoryid'],
                "from": start,
                "to": end}

    response = requests.request("POST", url, params=querystring).json()

    total_pages = math.ceil((response['totalCount'] - 2*response['pageSize'])/response['pageSize'])
    items = response['items']
    for i in items:
        i['eventid_from_url'] = int(i['url'].strip('/').split('/')[-1])
    item_dicts = []
    with tqdm(total=total_pages) as pbar:
        while response['remaining'] > 0:
            item_dicts.append(items)

            querystring['pageIndex'] = int(querystring['pageIndex']) + 1
            response = requests.request("POST", url, params=querystring).json()
            items = response['items']
            for i in items:
                i['eventid_from_url'] = int(i['url'].strip('/').split('/')[-1])
            pbar.update(1)
            
        # append last page
        item_dicts.append(items)

    return list(itertools.chain.from_iterable(item_dicts))

In [2]:
evs = get_events('NFL')

8it [00:02,  2.73it/s]                       


In [3]:
evs[0]

{'eventId': 150021223,
 'name': 'Philadelphia Eagles at Houston Texans',
 'url': '/houston-texans-houston-tickets-11-3-2022/event/105255775/',
 'dayOfWeek': 'Thu',
 'formattedDate': 'Nov 03',
 'formattedTime': '7:15 PM',
 'formattedDateWithoutYear': 'Nov 03',
 'isTbd': False,
 'isDateConfirmed': True,
 'isTimeConfirmed': True,
 'venueName': 'NRG Stadium',
 'formattedVenueLocation': 'Houston, TX, USA',
 'countryCode': 'US',
 'formattedMinPrice': '$40',
 'userDistanceFromVenue': 2166435.96663194,
 'formattedDistanceInMiles': '',
 'formattedRescheduledFromDate': '',
 'eventCountdownMessage': 'Tomorrow',
 'timeToEventTotalMilliseconds': 0,
 'hasActiveListings': True,
 'eventid_from_url': 105255775}

In [4]:
event_path = '/detroit-lions-detroit-tickets-9-18-2022/event/105255466/'
url = f"https://www.stubhub.com/event/105255466"
response = requests.request("POST", url)
response = requests.request("POST", response.url)
response

    


<Response [404]>

In [5]:
class EnhancedSession(requests.Session):
    '''
    Set timeout at the session level .
    '''
    def __init__(self, timeout=(3.05, 4)):
        self.timeout = timeout
        return super().__init__()

    def request(self, method, url, **kwargs):
        # print("EnhancedSession request")
        if "timeout" not in kwargs:
            kwargs["timeout"] = self.timeout
        return super().request(method, url, **kwargs)

def prepare_session():
    '''
    Prepare requests session with default timeout and retry.
    '''
    s = EnhancedSession(timeout=10)
    retries = Retry(total=5,
                    backoff_factor=0.1,
                    method_whitelist=["POST", "GET"])
    s.mount('http://', HTTPAdapter(max_retries=retries))
    s.mount('https://', HTTPAdapter(max_retries=retries))
    return s

def get_listings(eventid):
    '''
    Get all event listings.
    '''
    s = prepare_session()
    
    url = f"https://www.stubhub.com/event/{eventid}"
    url = s.request("POST", url,allow_redirects=True).url
    response = s.request("POST", url)


    if response.status_code == 404:
        print('404, Probably, no listings for event {}'.format(eventid))
        # print(url)
        return None

    respjson= response.json()
    querystring = {"CurrentPage":"1"}
    total_pages = math.ceil((respjson['TotalCount'] - 2*respjson['PageSize'])/respjson['PageSize'])
    items = respjson['Items']
    item_dicts = []
    with tqdm(total=total_pages, desc=f'Event Id: {eventid}', leave=False) as pbar:
        while respjson['ItemsRemaining'] > 0:
            item_dicts.append(items)

            querystring['CurrentPage'] = int(querystring['CurrentPage']) + 1
            respjson = s.request("POST", url, params=querystring).json()
            items = respjson['Items']
            pbar.set_description(f'Code: {response.status_code} Event Id: {eventid}')
            pbar.update(1)
            
        # append last page
        item_dicts.append(items)

    return list(itertools.chain.from_iterable(item_dicts))

def parse_listing(l):
    '''
    Parse listing by extracting relevant columns.
    '''
    listing = {}
    for c in columns:
        listing[c] = columns[c]

In [6]:
listings = get_listings(150021320)

In [7]:
listings[0]

{'RatingMessage': None,
 'RatingValue': 0.0,
 'Id': 5586993071,
 'EventId': 150021320,
 'CssPostFix': 'blue2',
 'TraderType': None,
 'Section': 'Section 309',
 'SectionId': 701727,
 'SectionMapName': 's309',
 'RowMapName': '',
 'Row': '',
 'Price': '$343',
 'PriceWithFees': '$447',
 'RawPrice': 342.54,
 'QuantityRange': '1',
 'IsAlreadySold': None,
 'IsAlreadySoldTicketClass': None,
 'LastInSectionString': None,
 'LastInSectionDescriptionString': None,
 'AboutToSellOut': None,
 'AboutToSellOutExplanation': None,
 'NumberOfTicketsString': None,
 'AdmitXTicketsString': None,
 'TicketsLeftInListingMessage': {'Message': '1 ticket remaining',
  'Qualifier': 'in this listing',
  'Disclaimer': None,
  'HasValue': True,
  'FeatureTrackingKey': '2fbb0992-0355-42c8-98d2-4f0616842abb'},
 'ShowNumberOfTicketsString': False,
 'LastTicketsInSectionMessage': {'Message': None,
  'Qualifier': None,
  'Disclaimer': None,
  'HasValue': False,
  'FeatureTrackingKey': None},
 'NumberOfTicketsDisplayed': No

In [8]:
ids = [a['Id'] for a in listings]
len(ids)

487

In [9]:

DATA_FOLDER = 'data/nba'
LEAGUE = 'NBA'

# get list of events if does not exist
event_files = list(reversed(sorted([f for f in os.listdir(DATA_FOLDER) if 'events' in f])))
if len(event_files) > 0:
    with open(os.path.join(DATA_FOLDER,event_files[0])) as f:
        evs = json.load(f)
else:
    evs = get_events(LEAGUE)
    with open(os.path.join(DATA_FOLDER,f'events_{int(time.time())}.json'), 'w+') as f:
        json.dump(evs,f)

# get listings
retrieve_time = int(time.time())
for e in tqdm(evs):
    listings = get_listings(e['eventId'])
    if listings == None or len(listings) == 0:
        listings = get_listings(e['eventid_from_url'])
    if listings != None and len(listings) > 0:
        with open(os.path.join(DATA_FOLDER,f'listings/{e["eventId"]}_{retrieve_time}.json'), 'w', encoding='utf-8') as f:
            json.dump(listings, f, ensure_ascii=False)
        

  0%|          | 0/1092 [00:00<?, ?it/s]

404, Probably, no listings for event 150341911
https://www.stubhub.com/chicago-bulls-tickets/performer/2863/


  0%|          | 1/1092 [00:01<32:38,  1.79s/it]

404, Probably, no listings for event 150341911
https://www.stubhub.com/chicago-bulls-tickets/performer/2863/
404, Probably, no listings for event 150338033
https://www.stubhub.com/cleveland-cavaliers-tickets/performer/2987/


  0%|          | 2/1092 [00:04<37:56,  2.09s/it]

...

  1%|          | 8/1092 [00:15<31:33,  1.75s/it]

404, Probably, no listings for event 150337988
https://www.stubhub.com/new-york-knicks-tickets/performer/2742/
404, Probably, no listings for event 150338256
https://www.stubhub.com/miami-heat-tickets/performer/2745/


  1%|          | 9/1092 [00:16<32:02,  1.77s/it]

404, Probably, no listings for event 150338256
https://www.stubhub.com/miami-heat-tickets/performer/2745/


  2%|▏         | 17/1092 [00:36<32:15,  1.80s/it]  

In [89]:

DATA_FOLDER = 'data/nfl'
LEAGUE = 'NFL'

# get list of events if does not exist
event_files = list(reversed(sorted([f for f in os.listdir(DATA_FOLDER) if 'events' in f])))
if len(event_files) > 0:
    with open(os.path.join(DATA_FOLDER,event_files[0])) as f:
        evs = json.load(f)
else:
    evs = get_events(LEAGUE)
    with open(os.path.join(DATA_FOLDER,f'events_{int(time.time())}.json'), 'w+') as f:
        json.dump(evs,f)

# get listings
retrieve_time = int(time.time())
for e in tqdm(evs):
    listings = get_listings(e['eventId'])
    if listings == None or len(listings) == 0:
        listings = get_listings(e['eventid_from_url'])
    if listings != None and len(listings) > 0:
        with open(os.path.join(DATA_FOLDER,f'listings/{e["eventId"]}_{retrieve_time}.json'), 'w', encoding='utf-8') as f:
            json.dump(listings, f, ensure_ascii=False)
        

  4%|▎         | 6/166 [00:19<08:38,  3.24s/it]

404, Probably, no listings for event 9525140
https://www.stubhub.com/event/9525140


  5%|▌         | 9/166 [00:36<12:26,  4.76s/it]

404, Probably, no listings for event 9681631
https://www.stubhub.com/event/9681631


  8%|▊         | 13/166 [01:02<16:43,  6.56s/it]

...

 97%|█████████▋| 161/166 [32:46<02:06, 25.37s/it]

404, Probably, no listings for event 9584042
https://www.stubhub.com/event/9584042


 99%|█████████▉| 164/166 [33:30<00:36, 18.48s/it]

404, Probably, no listings for event 9583509
https://www.stubhub.com/event/9583509


100%|██████████| 166/166 [33:46<00:00, 12.21s/it]


In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
df = pd.read_json('data/nba/listings/150339092_1667282312.json')
df.describe()

,RatingMessage,RatingValue,Id,EventId,TraderType,SectionId,RawPrice,IsAlreadySold,IsAlreadySoldTicketClass,LastInSectionString,LastInSectionDescriptionString,AboutToSellOut,AboutToSellOutExplanation,NumberOfTicketsString,AdmitXTicketsString,NumberOfTicketsDisplayed,LocationRank,ListingRank,PriceRank,SectionColor,ShowStandOrSitFreely,PreAllocatedSeats,PriceBucketColourClass,GeneralAdmissionExplanation,FaceValueText,MaxQuantity,SourceId,TicketClass,PackageTitle,Package,Seats,RowId,TicketReservationContent,IsCheapestInSection,PeopleViewedMessage,HottestTicketMessage,PeopleViewingMessage,TicketTypeId,TicketTypesWithIcons,TicketTypeForPageEvent,ShowClickAndDownloadMessage,XOtherInterestedInBadgeText,XOthersInterestedInTicketsMessage,ListingNoteSectionTitle,NowAtPrice,othersViewingTheseTicketsMessage,CountryCode,HumanFilterIds,RestrictionListingNotes,RecommendReason,RecentlyAddedString,RecentlySoldTicketCountString,SeatInfo,LocationRanking,LocationRankingMessage,AlternativeListingMessage,MostPopularSectionDescription,ViewFromSectionUrl,ViewFromSeatUrl,ViewFromSectionDisclaimer,DistinctPriceTicketContent,MaySoonChangePriceMessage,CheapestInSectionMessage,SectionSellingFastMessage,SeasonalDealMessage,SeasonalDealClass,SeasonalDealExplanation,MonthlyDealMessage,MonthlyDealDescription,SellerIsNearbyMessage,XPercentageOfPeopleBoughtThisSection,XPercentageOfPeopleBoughtThisSectionExplanation,EmailDeliveryMessage,ExpiresMessage,ExpiresMessageDisclaimer,AlternativeOffer,SectionRecentlySoldCount,PipelineEntryMessage,DisplayPrice,LockedTimeSeconds,SectionLikelyToSellOutMessage,CheapestInCityMessage,FaceValueMessage,FvTrackingKey,InvalidSeatsTrackingKey
count,0.0,1313.0,1.313000e+03,1313.0,0.0,1.300000e+03,1313.000000,189.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1313.0,1313.0,1313.0,0.0,0.0,0.0,0.0,0.0,0.0,1313.000000,0.0,1313.000000,0.0,0.0,0.0,1.300000e+03,0.0,0.0,0.0,0.0,0.0,1313.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1313.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1313.0,0.0,1313.000000,1313.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,0.0,5.544865e+09,150339092.0,NaN,1.217728e+06,291.337281,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.718203,NaN,2328.449353,NaN,NaN,NaN,8.629315e+05,NaN,NaN,NaN,NaN,NaN,9.486672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,291.316070,0.0,NaN,NaN,NaN,NaN,NaN
std,NaN,0.0,6.759717e+07,0.0,NaN,1.559571e+05,417.806801,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.636506,NaN,2071.654580,NaN,NaN,NaN,5.453635e+05,NaN,NaN,NaN,NaN,NaN,1.781086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,417.805699,0.0,NaN,NaN,NaN,NaN,NaN
min,NaN,0.0,5.442922e+09,150339092.0,NaN,1.132924e+06,31.500000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,0.000000,NaN,NaN,NaN,5.412190e+05,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,32.000000,0.0,NaN,NaN,NaN,NaN,NaN
25%,NaN,0.0,5.462843e+09,150339092.0,NaN,1.133336e+06,133.250000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,954.000000,NaN,NaN,NaN,5.420215e+05,NaN,NaN,NaN,NaN,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,133.000000,0.0,NaN,NaN,NaN,NaN,NaN
50%,NaN,0.0,5.580010e+09,150339092.0,NaN,1.133534e+06,200.900000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,1687.000000,NaN,NaN,NaN,5.432095e+05,NaN,NaN,NaN,NaN,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [11]:
with open('data/nba/listings/150339092_1667282312.json') as f: 
    j = json.load(f)
j[0]

{'RatingMessage': None,
 'RatingValue': 0.0,
 'Id': 5575812083,
 'EventId': 150339092,
 'CssPostFix': '',
 'TraderType': None,
 'Section': 'PARKINGING',
 'SectionId': None,
 'SectionMapName': '',
 'RowMapName': '$r',
 'Row': None,
 'Price': '$32',
 'PriceWithFees': '$40',
 'RawPrice': 31.5,
 'QuantityRange': '1',
 'IsAlreadySold': None,
 'IsAlreadySoldTicketClass': None,
 'LastInSectionString': None,
 'LastInSectionDescriptionString': None,
 'AboutToSellOut': None,
 'AboutToSellOutExplanation': None,
 'NumberOfTicketsString': None,
 'AdmitXTicketsString': None,
 'TicketsLeftInListingMessage': {'Message': '1 ticket remaining',
  'Qualifier': 'in this listing',
  'Disclaimer': None,
  'HasValue': True,
  'FeatureTrackingKey': '2fbb0992-0355-42c8-98d2-4f0616842abb'},
 'ShowNumberOfTicketsString': False,
 'LastTicketsInSectionMessage': None,
 'NumberOfTicketsDisplayed': None,
 'WillBeAnimated': False,
 'LocationRank': 0.0,
 'ListingRank': 0.0,
 'PriceRank': 0.0,
 'SectionColor': None,
 'Ha

In [7]:
import os
dfs = [] # an empty list to store the data frames
file_list = [f for f in os.listdir('data/nba/listings/') if '.json' in f]
for file in file_list:
    print(file)
    data = pd.read_json('data/nba/listings/{}'.format(file)) # read data frame from json file
    dfs.append(data) # append the data frame to the list

temp = pd.concat(dfs, ignore_index=True)
temp

150338343_1667282312.json


NameError: name 'pd' is not defined

In [24]:
temp.describe()

,RatingMessage,RatingValue,Id,EventId,TraderType,SectionId,RawPrice,IsAlreadySold,IsAlreadySoldTicketClass,LastInSectionString,LastInSectionDescriptionString,AboutToSellOut,AboutToSellOutExplanation,NumberOfTicketsString,AdmitXTicketsString,NumberOfTicketsDisplayed,LocationRank,ListingRank,PriceRank,SectionColor,ShowStandOrSitFreely,PreAllocatedSeats,PriceBucketColourClass,GeneralAdmissionExplanation,FaceValueText,MaxQuantity,SourceId,TicketClass,PackageTitle,Package,Seats,RowId,TicketReservationContent,IsCheapestInSection,PeopleViewedMessage,HottestTicketMessage,PeopleViewingMessage,TicketTypeId,TicketTypesWithIcons,TicketTypeForPageEvent,ShowClickAndDownloadMessage,XOtherInterestedInBadgeText,ListingNoteSectionTitle,NowAtPrice,othersViewingTheseTicketsMessage,CountryCode,HumanFilterIds,RestrictionListingNotes,RecommendReason,RecentlyAddedString,RecentlySoldTicketCountString,SeatInfo,LocationRanking,LocationRankingMessage,AlternativeListingMessage,MostPopularSectionDescription,ViewFromSectionUrl,ViewFromSeatUrl,ViewFromSectionDisclaimer,DistinctPriceTicketContent,MaySoonChangePriceMessage,CheapestInSectionMessage,SectionSellingFastMessage,SeasonalDealMessage,SeasonalDealClass,SeasonalDealExplanation,MonthlyDealMessage,MonthlyDealDescription,SellerIsNearbyMessage,XPercentageOfPeopleBoughtThisSection,XPercentageOfPeopleBoughtThisSectionExplanation,EmailDeliveryMessage,ExpiresMessage,ExpiresMessageDisclaimer,AlternativeOffer,SectionRecentlySoldCount,PipelineEntryMessage,DisplayPrice,LockedTimeSeconds,FaceValueMessage,FvTrackingKey,InvalidSeatsTrackingKey
count,0.0,381563.0,3.815630e+05,3.815630e+05,0.0,3.794720e+05,381563.000000,27131.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,381563.0,381563.0,381563.0,0.0,0.0,0.0,0.0,0.0,0.0,381563.000000,0.0,381563.000000,0.0,0.0,0.0,3.794720e+05,0.0,0.0,0.0,0.0,0.0,381563.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,381563.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,381563.0,0.0,381563.000000,381563.0,0.0,0.0,0.0
mean,NaN,0.0,5.538428e+09,1.503446e+08,NaN,7.195838e+05,285.681312,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,3.785446,NaN,1528.491410,NaN,NaN,NaN,1.211120e+06,NaN,NaN,NaN,NaN,NaN,8.198719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,285.674942,0.0,NaN,NaN,NaN
std,NaN,0.0,6.146614e+07,2.051117e+04,NaN,5.867632e+05,1123.550352,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.450560,NaN,2385.050357,NaN,NaN,NaN,6.615392e+05,NaN,NaN,NaN,NaN,NaN,3.723734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1123.551640,0.0,NaN,NaN,NaN
min,NaN,0.0,5.442877e+09,1.502468e+08,NaN,6.718500e+04,3.490000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,0.000000,NaN,NaN,NaN,3.100600e+04,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,3.000000,0.0,NaN,NaN,NaN
25%,NaN,0.0,5.475478e+09,1.503383e+08,NaN,1.491440e+05,63.210000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,407.000000,NaN,NaN,NaN,5.430610e+05,NaN,NaN,NaN,NaN,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,63.000000,0.0,NaN,NaN,NaN
50%,NaN,0.0,5.551211e+09,1.503388e+08,NaN,2.780550e+05,135.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,594.000000,NaN,NaN,NaN,1.327669e+06,NaN,NaN,NaN,NaN,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0

In [6]:
temp['RowMapName'].unique()

NameError: name 'temp' is not defined

In [ ]:
{'RatingMessage': None,
 'RatingValue': 0.0,
 'Id': 5575812083,
 'EventId': 150339092,
 'Section': 'PARKINGING',
 'SectionId': None,
 'SectionMapName': '',
 'RowMapName': '$r',
 'Row': None,
 'Price': '$32',
 'PriceWithFees': '$40',
 'RawPrice': 31.5,
 'QuantityRange': '1',
 'MaxQuantity': 1,
 'AvailableQuantities': [1],
 'TicketTypeNotes': '',
 'TicketClass': 0,
 'TicketClassName': None,
 'BuyUrl': '/secure/buy/Initialise?ListingID=5575812083&EventID=150339092&CategoryId=5973',
 'RowId': None,
 'TicketTypeId': 10,
 'RowContent': 'Row',
 'DisplayPrice': 32.0,
 'SellerTrackingKey': 'e205966f-497c-4d4f-b5f6-6be10d8e63ba'
  }